In [1]:
import Io_tf_binary_general as io

Welcome to JupyROOT 6.15/01


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import read_tree as rt

In [15]:
sciezka_sygnalu="data_rootowe_obrobione/MT_GluGlu.root"
sciezka_tla="data_rootowe_obrobione/MT_WJets.root"

In [3]:
def wczytaj_do_pythonowych(sciezka_rootowego):
    """funkcja zwraca pythonowe w wygodnej do uzycia formie"""
    legs, jets, global_params, properties = rt.read_tree(file_name=sciezka_rootowego)
    #te rzeczy sa zle, bo maja niewygodny format oraz global_params["nJets30"] ma
    #zdublowane dane, to znaczy [1,1,3,3,5,5 ...]
    #mozna to sprawdzic ta metoda
    #nJets30 ma od 0 do 6 a nawet do 8,
    #leg_1_charge ma -1 oraz 1
    #leg_2_charge ma -1 oraz 1
    #'leg_2_decayMode' ma 0,1,10
    kategoryczne=["nJets30","leg_1_charge","leg_2_charge",'leg_2_decayMode']
    zdublowany_klucz="nJets30"
    def czy_zdublowane(tab):
        for i in range(len(tab)//2):
            if tab[2*i] != tab[2*i+1]:
                print ("kontrprzylad dla tezy, ze to jest zdublowana lista")
                assert False
    czy_zdublowane(global_params[zdublowany_klucz])
    def wyintuj_kategoryczne(slownik):
        for k in slownik.keys():
            if k in kategoryczne:
                slownik[k]=np.array(slownik[k],dtype=int)
    wyintuj_kategoryczne(global_params)
    wyintuj_kategoryczne(properties)
    odrzucane_rzeczy=["BJetBetaScore"]
    #jets 1 jest zle, wszystkie dane tam sa stale chyba
    del jets[1]
    def wyrzuc_odrzucone(slownik):
        klucze=list(slownik.keys())
        for k in klucze:
            if k in odrzucane_rzeczy:
                del slownik[k]
    wyrzuc_odrzucone(global_params)
    wyrzuc_odrzucone(properties)
    do_zlogarytmowania_z_dodaniem_0_01=["higgsMassTrans","higgsPT","visMass",
                                        "leg_1_combreliso","leg_2_byCombinedIsolationDeltaBetaCorrRaw3Hits"]
    def zlogarytmoj(slownik):
        for k in slownik.keys():
            if k in do_zlogarytmowania_z_dodaniem_0_01:
                slownik[k]=np.log(np.array(slownik[k])+0.01)
    zlogarytmoj(global_params)
    zlogarytmoj(properties)
    def zlogarytmoj_energie(tab):
        for i in range(len(tab)):
            tab[i][0]=np.log(np.array(tab[i][0])+0.01)
    zlogarytmoj_energie(legs)
    zlogarytmoj_energie(jets)
    def f1(x):
        return np.log((1.-x))
    do_zrobienia_f1=["leg_2_byIsolationMVArun2v1DBoldDMwLTraw"]
    def zrob_f1(slownik):
        for k in slownik.keys():
            if k in do_zrobienia_f1:
                slownik[k]=f1(np.array(slownik[k]))
    zrob_f1(global_params)
    zrob_f1(properties)
    def przerob_legs_lub_jets(tab):
        tab=np.array(tab)
        return np.transpose(tab,(2,0,1))
    def przerob_global_params_lub_properties(slownik):
        wyrzut=[]
        jeden_z_kluczy=list(slownik.keys())[0]
        #uwaga hakerstwo
        liczba_przypadkow=0
        if jeden_z_kluczy==zdublowany_klucz:
            liczba_przypadkow=len(slownik[jeden_z_kluczy]) //2
        else:
            liczba_przypadkow=len(slownik[jeden_z_kluczy])
        for i in range(liczba_przypadkow):
            doczepiany={}
            for k in slownik.keys():
                if k != zdublowany_klucz:
                    doczepiany[k]=slownik[k][i]
                else:
                    doczepiany[k]=slownik[k][2*i]
            wyrzut.append(doczepiany)
        return wyrzut
    legs=przerob_legs_lub_jets(legs)
    jets=przerob_legs_lub_jets(jets)
    global_params=przerob_global_params_lub_properties(global_params)
    properties=przerob_global_params_lub_properties(properties)
    return legs, jets, global_params, properties


In [4]:
legs_syg, jets_syg, global_params_syg, properties_syg=\
wczytaj_do_pythonowych(sciezka_sygnalu)

[ML]	Opening root file for python conversion.
[ML]	Reading data from TTree.
[ML]	Conversion to python successful!


In [5]:
legs_tlo, jets_tlo, global_params_tlo, properties_tlo=\
wczytaj_do_pythonowych(sciezka_tla)

[ML]	Opening root file for python conversion.
[ML]	Reading data from TTree.
[ML]	Conversion to python successful!


In [6]:
liczba_przypadkow_tlo=len(legs_tlo)
liczba_przypadkow_syg=len(legs_syg)

In [7]:
print(liczba_przypadkow_tlo)
print(liczba_przypadkow_syg)

3087
59083


In [8]:
#mozna tu zauwazyc pewna dysproporcje. 
#na szybko mozna ta sytuacje rozwiazac uzywajac wielokrotnie przypadkow tla, tak by
#bylo mniej wiecej po rowno tych rzeczy

In [9]:
zapisywacz_syg=io.Io_tf_binary_general("sygnal",'w')

In [10]:
zapisywacz_tlo=io.Io_tf_binary_general("tlo",'w')

In [11]:
for i in range(liczba_przypadkow_syg):
    zapisywacz_syg.\
    write_from_tree(legs_syg[i],jets_syg[i],global_params_syg[i],
        properties_syg[i],1)

In [12]:
for i in range(liczba_przypadkow_tlo):
    zapisywacz_tlo.\
    write_from_tree(legs_tlo[i],jets_tlo[i],global_params_tlo[i],
        properties_tlo[i],0)

In [13]:
zapisywacz_syg.close()
zapisywacz_tlo.close()